# Robustness Analysis: Experimentation Notebook

### TO DO:

##### PRIO 1:
- there is an error in the bootstrap - it needs to keep links for all plants!

##### PRIO 2:
- can we change the output so that new analysis will add to existing files rather than rewrite them??
- add "trophic coherence" to the metric calculator

#### ----------------------------------------------------------------------------------------------------------------------------------

## Setup graph

In [2]:
import numpy as np
import pandas as pd
import sys
import networkx as nx
import random
import community as community_louvain

sys.path.append('../')

In [3]:
# Make Graph

# Define the path to your CSV file
csv_file_path = r"D:\PhD\data\chapter_2\scripts\foodweb-robustness-analysis-main\data\foodwebs\EA_Alpines.csv"

# Define the column names to use as source and target nodes
source_col = "Source_Name"  # Replace with the actual column name
target_col = "Target_Name"  # Replace with the actual column name

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_file_path, usecols=[source_col, target_col])

# Create a directed graph from the edge DataFrame
S = nx.from_pandas_edgelist(df, source=source_col, target=target_col, create_using=nx.DiGraph())
S = nx.reverse(S)

# Print some information about the subgraph
print(f"Number of nodes in the graph: {S.number_of_nodes()}")
print(f"Number of edges in the graph: {S.number_of_edges()}")
print("First ten nodes:", list(S.nodes())[:10])


Number of nodes in the graph: 9823
Number of edges in the graph: 428183
First ten nodes: ['Ovis gmelini musimon', 'Abies alba', 'Acer pseudoplatanus', 'Achillea millefolium', 'Adoxa moschatellina', 'Agrostis capillaris', 'Agrostis stolonifera', 'Ajuga reptans', 'Anthyllis vulneraria', 'Artemisia absinthium']


In [4]:
# Find isolated nodes by checking if both in-degree and out-degree are 0
isolated_nodes = [node for node in S.nodes() if S.in_degree(node) == 0 and S.out_degree(node) == 0]
print(f"Isolated nodes: {isolated_nodes}")

Isolated nodes: []


# Calculate which groups are in which trophic level

#### ----------------------------------------------------------------------------------------------------------------------------------------
## Get a bootstrap sample


In [5]:
def classify_trophic_categories(graph):
    # Initialize dictionaries to hold species by trophic level
    trophic_categories = {"Basal": [], "Primary": [], "Secondary": [], "Omnivores": []}
    
    basal_species = [node for node in graph.nodes() if graph.in_degree(node) == 0]
    trophic_categories["Basal"] = basal_species

    all_species = set(graph.nodes())
    non_basal_species = all_species - set(basal_species)
    
    for node in non_basal_species:
        predecessors = set(graph.predecessors(node))
        
        if predecessors.issubset(basal_species):
            # Primary consumers feed exclusively on Basal species
            trophic_categories["Primary"].append(node)
        elif not predecessors.isdisjoint(basal_species) and not predecessors.issubset(basal_species):
            # Omnivores consume both Basal and non-Basal
            trophic_categories["Omnivores"].append(node)
        else:
            # Secondary consumers feed exclusively on non-Basal species
            trophic_categories["Secondary"].append(node)
    
    trophic_categories["Primary"] += trophic_categories["Omnivores"]
    trophic_categories["Secondary"] += trophic_categories["Omnivores"]

    return trophic_categories

trophic_categories = classify_trophic_categories(S)

# Print out the classified species for verification
for level, species in trophic_categories.items():
    print(f"{level}: {len(species)} species")
    print(species[:10])  # Print only the first 10 species for brevity

Basal: 961 species
['Abies alba', 'Acer pseudoplatanus', 'Achillea millefolium', 'Adoxa moschatellina', 'Agrostis capillaris', 'Agrostis stolonifera', 'Ajuga reptans', 'Anthyllis vulneraria', 'Artemisia absinthium', 'Athyrium filix-femina']
Primary: 6488 species
['Arctophila superbiens', 'Hercostomus celer', 'Eupithecia cretaceata', 'Charissa ambiguata', 'Sterrhopterix standfussi', 'Meromyza variegata', 'Thricops longipes', 'Agrilus guerini', 'Pammene fasciana', 'Chamaesphecia aerifrons']
Secondary: 3795 species
['Leuciscus leuciscus', 'Hydroporus sabaudus', 'Dinaraea angustula', 'Drosophila melanogaster', 'Glischrochilus fasciatus', 'Sympetrum pedemontanum', 'Myzia oblongoguttata', 'Troglodytes troglodytes', 'Chrysis consanguinea', 'Buteo rufinus']
Omnivores: 1421 species
['Saphanus piceus', 'Rhinusa herbarum', 'Formica exsecta', 'Neophytobius muricatus', 'Sicus ferrugineus', 'Graphomya maculata', 'Nasonovia dasyphylli', 'Issus coleoptratus', 'Coturnix coturnix', 'Aphis epilobii']


In [6]:
species_to_find = "Gypaetus barbatus"

# Check if the species is in any of the trophic categories
found_category = None
for category, species_list in trophic_categories.items():
    if species_to_find in species_list:
        found_category = category
        break

if found_category is not None:
    print(f"The species '{species_to_find}' belongs to the trophic category: {found_category}")
else:
    print(f"The species '{species_to_find}' does not belong to any trophic category.")


The species 'Gypaetus barbatus' belongs to the trophic category: Secondary


In [7]:
frac = 0.05
bas_frac = int(frac * len(trophic_categories['Basal']))
prim_frac = int(frac * len(trophic_categories['Primary']))-int(frac * len(trophic_categories['Omnivores']))
top_frac = int(frac * len(trophic_categories['Secondary']))

if bas_frac+prim_frac+top_frac == int(S.number_of_nodes()*frac):
    print(f"Fracs equal")

print(f"Basal frac: {bas_frac}")
print(f"Primary frac: {prim_frac}")
print(f"Secondary frac: {top_frac}")


Basal frac: 48
Primary frac: 253
Secondary frac: 189


In [8]:
#here we're just testing to make sure that there are paths between basal + primary and secondary
bas_prim = set(trophic_categories['Basal']+trophic_categories['Primary']+trophic_categories['Omnivores'])
B = S.subgraph(bas_prim).copy()
# Find isolated nodes by checking if both in-degree and out-degree are 0
isolated_nodes = [node for node in B.nodes() if B.in_degree(node) == 0 and B.out_degree(node) == 0]
print(f"# nodes: {len(bas_prim)}")
print(f"# isolated nodes: {len(isolated_nodes)}")

# Initialize a dictionary to store trophic categories of isolated nodes
isolated_nodes_trophic_categories = {}

# Iterate through each isolated node
for node in isolated_nodes:
    # Check the trophic category of the node
    for category, nodes in trophic_categories.items():
        if node in nodes:
            isolated_nodes_trophic_categories[node] = category
            break  # Exit loop once trophic category is found

# Print the trophic categories of isolated nodes
for node, category in isolated_nodes_trophic_categories.items():
    print(f"Node: {node}, Trophic Category: {category}")

# TO DO: We need primary and secondary to be included here. that means that we choose primary from primary and secondary, and we choose secondary from apex, and we choose apex based on any of the ones below, and apex is based on eats anything but has no predators!


# nodes: 7449
# isolated nodes: 0


In [9]:
def connect_upstream(graph, downstream_nodes, upstream_category, upstream_frac):
    selected_upstream = set()

    for downstream_node in downstream_nodes:        

        # Correctly find potential connections that are not already selected
        potential_connections = [neighbor for neighbor in graph.successors(downstream_node)
                                 if neighbor in trophic_categories[upstream_category] or
                                 neighbor in trophic_categories["Omnivores"] and
                                 neighbor not in selected_upstream]
        if not potential_connections:
            # Log or handle the case where no potential connections are found
            print(f"No successors found for downstream node {downstream_node}")
            continue

        if potential_connections and len(selected_upstream) < upstream_frac:
            selected_node = random.choice(potential_connections)
            selected_upstream.add(selected_node)

    # Adjust for additional_needed based on upstream_frac, not prim_frac
    if len(selected_upstream) < upstream_frac:
        additional_needed = upstream_frac - len(selected_upstream)
        available_upstream = set(trophic_categories[upstream_category]) - selected_upstream

        if available_upstream:  # Check if there are available upstream nodes
            additional_selected = random.sample(list(available_upstream), min(additional_needed, len(available_upstream)))
            selected_upstream.update(additional_selected)

    return list(selected_upstream)

sampled_basal = random.sample(trophic_categories['Basal'], bas_frac)
sampled_primary = connect_upstream(S, sampled_basal, "Primary", prim_frac)
sampled_secondary = connect_upstream(S, sampled_primary, "Secondary", top_frac)
print(f"Selected basal: {len(sampled_basal)}")
print(f"Selected primary consumers: {len(sampled_primary)}")
print(f"Selected secondary consumers: {len(sampled_secondary)}")

all_sampled_nodes = set(sampled_secondary+sampled_primary+sampled_basal)
B = S.subgraph(all_sampled_nodes).copy()

isolated_nodes = [node for node in B.nodes() if B.in_degree(node) == 0 and B.out_degree(node) == 0]
print(f"# nodes: {len(B)}")
print(f"# isolated nodes: {len(isolated_nodes)}")

# Initialize a dictionary to store trophic categories of isolated nodes
isolated_nodes_trophic_categories = {}

# Iterate through each isolated node
for node in isolated_nodes:
    # Check the trophic category of the node
    for category, nodes in trophic_categories.items():
        if node in nodes:
            isolated_nodes_trophic_categories[node] = category
            break  # Exit loop once trophic category is found

# Print the trophic categories of isolated nodes
for node, category in isolated_nodes_trophic_categories.items():
    print(f"Node: {node}, Trophic Category: {category}")

No successors found for downstream node Amblyptilia punctidactyla
Selected basal: 48
Selected primary consumers: 253
Selected secondary consumers: 189
# nodes: 476
# isolated nodes: 3
Node: Clanga pomarina, Trophic Category: Secondary
Node: Blatta orientalis, Trophic Category: Secondary
Node: Chimarra marginata, Trophic Category: Secondary


In [10]:
isolated_nodes_counts = []  # List to store the counts of isolated nodes

for seed in range(1000):  # Iterate over random seeds 0-999
    random.seed(seed)  # Set the random seed

    # Your existing code here
    sampled_basal = random.sample(trophic_categories['Basal'], bas_frac)
    sampled_primary = connect_upstream(S, sampled_basal, "Primary", prim_frac)
    sampled_secondary = connect_upstream(S, sampled_primary, "Secondary", top_frac)

    all_sampled_nodes = set(sampled_secondary+sampled_primary+sampled_basal)
    B = S.subgraph(all_sampled_nodes).copy()

    isolated_nodes = [node for node in B.nodes() if B.in_degree(node) == 0 and B.out_degree(node) == 0]
    isolated_nodes_counts.append(len(isolated_nodes))

# Count how many times len(isolated_nodes) is greater than 0
isolated_nodes_gt_zero_count = sum(1 for count in isolated_nodes_counts if count > 0)

# Calculate max and mean
max_length = max(isolated_nodes_counts)
mean_length = np.mean(isolated_nodes_counts)

print(f"Number of times len(isolated_nodes) is greater than 0: {isolated_nodes_gt_zero_count}")
print(f"Max length of isolated_nodes: {max_length}")
print(f"Mean length of isolated_nodes: {mean_length}")


No successors found for downstream node Pyrausta purpuralis
No successors found for downstream node Cleptes nitidulus
No successors found for downstream node Pyrausta purpuralis
No successors found for downstream node Coleophora albitarsella
No successors found for downstream node Cleptes nitidulus
No successors found for downstream node Amblyptilia punctidactyla
No successors found for downstream node Coleophora albitarsella
No successors found for downstream node Trifurcula headleyella
No successors found for downstream node Amblyptilia punctidactyla
No successors found for downstream node Trifurcula headleyella
No successors found for downstream node Physocephala rufipes
No successors found for downstream node Amblyptilia punctidactyla
No successors found for downstream node Coleophora albitarsella
No successors found for downstream node Amblyptilia punctidactyla
No successors found for downstream node Pyrausta purpuralis
No successors found for downstream node Amblyptilia punctidac

In [254]:
node = "Gypaetus barbatus"  # Example node
successors = list(S.successors(node))
print("S Successors:", successors)

S Successors: ['Corvus corax', 'Corvus corone', 'Bubo bubo', 'Aquila chrysaetos']


In [ ]:

#this is just for the main code
# Create a new edge DataFrame from the sampled nodes
df = df[(df[source_col].isin(final_subset)) & (df[target_col].isin(final_subset))]
subset_graph = S.subgraph(final_subset).copy()


# Other

In [287]:
# Trophic coherency

# Trophic coherency calculation:
def trophic_coherency(graph, trophic_levels):
    coherency_sum = 0
    for node in graph.nodes():
        parent_nodes = list(graph.predecessors(node))
        if parent_nodes:
            parent_level = max(trophic_levels[parent] for parent in parent_nodes)
            coherency_sum += abs(trophic_levels[node] - parent_level)
    return coherency_sum / len(graph.nodes())


coherency_G = trophic_coherency(S, nx.algorithms.trophic_levels(S))
coherency_S = trophic_coherency(B, nx.algorithms.trophic_levels(B))
print(f"Graph Trophic Coherency: {coherency_G}")
print(f"Subgraph Trophic Coherency: {coherency_S}")



Graph Trophic Coherency: 0.47700835554371135
Subgraph Trophic Coherency: 0.41292894011387743
